In [1]:
import math
#numpy is numerical python, which lets us do math!
import numpy as np
#pandas is for reading in and manipulating dataframe
import pandas as pd
#matplotlib is a plotting library the originated in matlab
from matplotlib import pyplot as plt
#seaborn wraps around matplotlib so that we can make pretty plots more easliy
import seaborn as sns
import missingno as msno
#this little line of code lets us see the plots in the notebook
%matplotlib inline
#this uses seaborn (sns) to set the style for all the plots
sns.set(context='poster', style='whitegrid', font_scale=1.4)
from os.path import join
from scipy.stats import ttest_ind, spearmanr, pearsonr, ttest_rel, kstest
import statsmodels as sm
from sklearn.linear_model import LinearRegression, Lasso
import fancyimpute as fi

#less important for plotting
from glob import glob
import statsmodels.api as sm
from statsmodels.iolib.summary2 import summary_col
from statsmodels.sandbox.stats.multicomp import multipletests

/Users/katherine/anaconda/lib/python2.7/site-packages/pandas/computation/__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used

  UserWarning)


ImportError: No module named fancyimpute

In [ ]:
import pandas
pandas.__version__

In [ ]:
def calculate_pvalues(df, correlation):
    from scipy.stats import pearsonr, pearsonr
    df = df.dropna()._get_numeric_data()
    dfcols = pd.DataFrame(columns=df.columns)
    pvalues = dfcols.transpose().join(dfcols, how='outer')
    for r in df.columns:
        for c in df.columns:
            if correlation == 'pearson':
                pvalues[r][c] = round(pearsonr(df[r], df[c])[1], 4)
            if correlation == 'spearman':
                pvalues[r][c] = round(spearmanr(df[r], df[c])[1], 4)
            else:
                raise ValueError('Invalid correlation method specified. Accpetable values are \'pearson\' and \'spearman\'.')
    return pvalues

In [ ]:
#Li & Ji (2005) method for multiple comparisons corrections
#calculating number of effective comparisons M_eff
def liji_sidak_mc(data, alpha):
    import math
    import numpy as np
    
    mc_corrmat = data.corr()
    eigvals, eigvecs = np.linalg.eig(mc_corrmat)

    M_eff = 0
    for eigval in eigvals:
        if abs(eigval) >= 0:
            if abs(eigval) >= 1:
                M_eff += 1
            else:
                M_eff += abs(eigval) - math.floor(abs(eigval))
        else:
            M_eff += 0
    print('Number of effective comparisons: {0}'.format(M_eff))

    #and now applying M_eff to the Sidak procedure
    sidak_p = 1 - (1 - alpha)**(1/M_eff)
    if sidak_p < 0.00001:
        print('Critical value of {:.3f}'.format(alpha),'becomes {:2e} after corrections'.format(sidak_p))
    else:
        print('Critical value of {:.3f}'.format(alpha),'becomes {:.6f} after corrections'.format(sidak_p))
    return sidak_p, M_eff

In [ ]:
fig_dir = '/Users/Katie/Dropbox/Projects/physics-retrieval/figures'

In [ ]:
data_dir = '/Users/Katie/Dropbox/Projects/physics-retrieval/data'
df = pd.read_csv(join(data_dir, 'rescored', 'non-brain-data.csv'), index_col=0, header=0)

df_f = df[df['Sex'] == 'F']
df_f = df_f.drop('Sex', axis=1)
df_m = df[df['Sex'] == 'M']
df_m = df_m.drop('Sex', axis=1)

df_f['const'] = 1
df_m['const'] = 1

In [ ]:
df.head()

In [ ]:
rsgt_df = pd.read_csv(join(data_dir, 'resting-state_graphtheory_shen+craddock_2019-05-29.csv'), 
                      index_col=0, header=0)
rsgt_df = pd.read_csv(join(data_dir, 'output', 'resting-state_graphtheory_shen+craddock_2019-11-20.csv'), 
                      index_col=0, header=0)
head_size = pd.read_csv(join(data_dir, 'head-size_2019-05-29 15:19:53.287525.csv'), index_col=0, header=0)
head_size['normalized head size'] = (head_size['average_head_size']-np.mean(head_size['average_head_size']))/np.std(head_size['average_head_size'])

In [ ]:
fd = pd.read_csv(join(data_dir, 'avg-fd-per-condition-per-run_2019-05-29.csv'), index_col=0, header=0)
fd['normalized fd'] = (fd['average fd']-np.mean(fd['average fd']))/np.std(fd['average fd'])
retr_fd = fd[fd['task'] == 'retr']
reas_fd = fd[fd['task'] == 'reas']
fci_fd = fd[fd['task'] == 'fci']

df_pivot = retr_fd[retr_fd['condition'] == 'high-level'].reset_index()
retr_phys_fd = df_pivot.pivot(index='subject', columns='session', values='average fd')
retr_phys_fd.rename({'pre': 'pre phys retr fd', 'post': 'post phys retr fd'}, axis=1, inplace=True)

df_pivot = retr_fd[retr_fd['condition'] == 'lower-level'].reset_index()
retr_genr_fd = df_pivot.pivot(index='subject', columns='session', values='average fd')
retr_genr_fd.rename({'pre': 'pre gen retr fd', 'post': 'post gen retr fd'}, axis=1, inplace=True)

df_pivot = reas_fd[reas_fd['condition'] == 'high-level'].reset_index()
reas_inf_fd = df_pivot.pivot(index='subject', columns='session', values='average fd')
reas_inf_fd.rename({'pre': 'pre infr reas fd', 'post': 'post infr reas fd'}, axis=1, inplace=True)

df_pivot = reas_fd[reas_fd['condition'] == 'lower-level'].reset_index()
reas_base_fd = df_pivot.pivot(index='subject', columns='session', values='average fd')
reas_base_fd.rename({'pre': 'pre base reas fd', 'post': 'post base reas fd'}, axis=1, inplace=True)

df_pivot = fci_fd[fci_fd['condition'] == 'high-level'].reset_index()
fci_phys_fd = df_pivot.pivot(index='subject', columns='session', values='average fd')
fci_phys_fd.rename({'pre': 'pre phys fci fd', 'post': 'post phys fci fd'}, axis=1, inplace=True)

df_pivot = fci_fd[fci_fd['condition'] == 'lower-level'].reset_index()
fci_ctrl_fd = df_pivot.pivot(index='subject', columns='session', values='average fd')
fci_ctrl_fd.rename({'pre': 'pre ctrl fci fd', 'post': 'post ctrl fci fd'}, axis=1, inplace=True)

In [ ]:
rest_fd = pd.read_csv(join(data_dir, 'avg-fd-per-run-rest_2019-05-31.csv'), index_col=0, header=0)
rs_fd = rest_fd.reset_index().pivot(index='subject', columns='session', values='average fd')
rs_fd.rename({'pre': 'pre rest fd', 'post': 'post rest fd'}, axis=1, inplace=True)

In [ ]:
fci_df

In [ ]:
fci_df.head()

In [14]:
fci_df = pd.read_csv(join(data_dir, 'physics-learning-fci_graphtheory_shen+craddock_2019-05-31.csv'), 
                     index_col=0, header=0)

#task_df = pd.read_csv(join(data_dir, 'output', 'physics-learning-tasks_graphtheory_shen+craddock2019-11-20.csv'), 
#                     index_col=0, header=0)
fci_df = task_df[task_df['task'] == 'fci']
phys_fci = fci_df[fci_df['condition'] == 'high-level']
ctrl_fci = fci_df[fci_df['condition'] == 'lower-level']

#physics fci measures per session per parcellation
df_pivot = phys_fci[phys_fci['mask'] == 'shen2015'].reset_index()
shen_eff_phys_fci_df = df_pivot.pivot(index='subject', columns='session', values='efficiency')
shen_eff_phys_fci_df.rename({0: 'pre phys fci shen eff', 1: 'post phys fci shen eff'}, axis=1, inplace=True)

shen_cpl_phys_fci_df = df_pivot.pivot(index='subject', columns='session', values='charpath')
shen_cpl_phys_fci_df.rename({0: 'pre phys fci shen cpl', 1: 'post phys fci shen cpl'}, axis=1, inplace=True)

shen_mod_phys_fci_df = df_pivot.pivot(index='subject', columns='session', values='modularity')
shen_mod_phys_fci_df.rename({0: 'pre phys fci shen mod', 1: 'post phys fci shen mod'}, axis=1, inplace=True)

df_pivot = phys_fci[phys_fci['mask'] == 'craddock2012'].reset_index()
crad_eff_phys_fci_df = df_pivot.pivot(index='subject', columns='session', values='efficiency')
crad_eff_phys_fci_df.rename({0: 'pre phys fci crad eff', 1: 'post phys fci crad eff'}, axis=1, inplace=True)

crad_cpl_phys_fci_df = df_pivot.pivot(index='subject', columns='session', values='charpath')
crad_cpl_phys_fci_df.rename({0: 'pre phys fci crad cpl', 1: 'post phys fci crad cpl'}, axis=1, inplace=True)

crad_mod_phys_fci_df = df_pivot.pivot(index='subject', columns='session', values='modularity')
crad_mod_phys_fci_df.rename({0: 'pre phys fci crad mod', 1: 'post phys fci crad mod'}, axis=1, inplace=True)


#ctrl fci measures per session per parcellation
df_pivot = ctrl_fci[ctrl_fci['mask'] == 'shen2015'].reset_index()
shen_eff_ctrl_fci_df = df_pivot.pivot(index='subject', columns='session', values='efficiency')
shen_eff_ctrl_fci_df.rename({0: 'pre ctrl fci shen eff', 1: 'post ctrl fci shen eff'}, axis=1, inplace=True)

shen_cpl_ctrl_fci_df = df_pivot.pivot(index='subject', columns='session', values='charpath')
shen_cpl_ctrl_fci_df.rename({0: 'pre ctrl fci shen cpl', 1: 'post ctrl fci shen cpl'}, axis=1, inplace=True)

shen_mod_ctrl_fci_df = df_pivot.pivot(index='subject', columns='session', values='modularity')
shen_mod_ctrl_fci_df.rename({0: 'pre ctrl fci shen mod', 1: 'post ctrl fci shen mod'}, axis=1, inplace=True)

df_pivot = ctrl_fci[ctrl_fci['mask'] == 'craddock2012'].reset_index()
crad_eff_ctrl_fci_df = df_pivot.pivot(index='subject', columns='session', values='efficiency')
crad_eff_ctrl_fci_df.rename({0: 'pre ctrl fci crad eff', 1: 'post ctrl fci crad eff'}, axis=1, inplace=True)

crad_cpl_ctrl_fci_df = df_pivot.pivot(index='subject', columns='session', values='charpath')
crad_cpl_ctrl_fci_df.rename({0: 'pre ctrl fci crad cpl', 1: 'post ctrl fci crad cpl'}, axis=1, inplace=True)

crad_mod_ctrl_fci_df = df_pivot.pivot(index='subject', columns='session', values='modularity')
crad_mod_ctrl_fci_df.rename({0: 'pre ctrl fci crad mod', 1: 'post ctrl fci crad mod'}, axis=1, inplace=True)

KeyError: 'efficiency'

In [ ]:
#task_df = pd.read_csv(join(data_dir, 'physics-learning-tasks_graphtheory_shen+craddock_2019-05-30.csv'), index_col=0, header=0)
retr_df = task_df[task_df['task'] == 'retr']
retr_phys_df = retr_df[retr_df['condition'] == 'high-level']
genr_phys_df = retr_df[retr_df['condition'] == 'lower-level']

#physics retrieval measures per session per parcellation
df_pivot = retr_phys_df[retr_phys_df['mask'] == 'shen2015'].reset_index()
shen_eff_retr_phys_df = df_pivot.pivot(index='subject', columns='session', values='efficiency')
shen_eff_retr_phys_df.rename({0: 'pre phys retr shen eff', 1: 'post phys retr shen eff'}, axis=1, inplace=True)

shen_cpl_retr_phys_df = df_pivot.pivot(index='subject', columns='session', values='charpath')
shen_cpl_retr_phys_df.rename({0: 'pre phys retr shen cpl', 1: 'post phys retr shen cpl'}, axis=1, inplace=True)

shen_mod_retr_phys_df = df_pivot.pivot(index='subject', columns='session', values='modularity')
shen_mod_retr_phys_df.rename({0: 'pre phys retr shen mod', 1: 'post phys retr shen mod'}, axis=1, inplace=True)

df_pivot = retr_phys_df[retr_phys_df['mask'] == 'craddock2012'].reset_index()
crad_eff_retr_phys_df = df_pivot.pivot(index='subject', columns='session', values='efficiency')
crad_eff_retr_phys_df.rename({0: 'pre phys retr crad eff', 1: 'post phys retr crad eff'}, axis=1, inplace=True)

crad_cpl_retr_phys_df = df_pivot.pivot(index='subject', columns='session', values='charpath')
crad_cpl_retr_phys_df.rename({0: 'pre phys retr crad cpl', 1: 'post phys retr crad cpl'}, axis=1, inplace=True)

crad_mod_retr_phys_df = df_pivot.pivot(index='subject', columns='session', values='modularity')
crad_mod_retr_phys_df.rename({0: 'pre phys retr crad mod', 1: 'post phys retr crad mod'}, axis=1, inplace=True)

#general retrieval measures per session per parcellation
df_pivot = genr_phys_df[genr_phys_df['mask'] == 'shen2015'].reset_index()
shen_eff_retr_genr_df = df_pivot.pivot(index='subject', columns='session', values='efficiency')
shen_eff_retr_genr_df.rename({0: 'pre gen retr shen eff', 1: 'post gen retr shen eff'}, axis=1, inplace=True)

shen_cpl_retr_genr_df = df_pivot.pivot(index='subject', columns='session', values='charpath')
shen_cpl_retr_genr_df.rename({0: 'pre gen retr shen cpl', 1: 'post gen retr shen cpl'}, axis=1, inplace=True)

shen_mod_retr_genr_df = df_pivot.pivot(index='subject', columns='session', values='modularity')
shen_mod_retr_genr_df.rename({0: 'pre gen retr shen mod', 1: 'post gen retr shen mod'}, axis=1, inplace=True)

df_pivot = genr_phys_df[genr_phys_df['mask'] == 'craddock2012'].reset_index()
crad_eff_retr_genr_df = df_pivot.pivot(index='subject', columns='session', values='efficiency')
crad_eff_retr_genr_df.rename({0: 'pre gen retr crad eff', 1: 'post gen retr crad eff'}, axis=1, inplace=True)

crad_cpl_retr_genr_df = df_pivot.pivot(index='subject', columns='session', values='charpath')
crad_cpl_retr_genr_df.rename({0: 'pre gen retr crad cpl', 1: 'post gen retr crad cpl'}, axis=1, inplace=True)

crad_mod_retr_genr_df = df_pivot.pivot(index='subject', columns='session', values='modularity')
crad_mod_retr_genr_df.rename({0: 'pre gen retr crad mod', 1: 'post gen retr crad mod'}, axis=1, inplace=True)

In [ ]:
reas_df = task_df[task_df['task'] == 'reas']
reas_infr_df = reas_df[reas_df['condition'] == 'high-level']
reas_base_df = reas_df[reas_df['condition'] == 'lower-level']

#infrics reasieval measures per session per parcellation
df_pivot = reas_infr_df[reas_infr_df['mask'] == 'shen2015'].reset_index()
shen_eff_reas_infr_df = df_pivot.pivot(index='subject', columns='session', values='efficiency')
shen_eff_reas_infr_df.rename({0: 'pre infr reas shen eff', 1: 'post infr reas shen eff'}, axis=1, inplace=True)

shen_cpl_reas_infr_df = df_pivot.pivot(index='subject', columns='session', values='charpath')
shen_cpl_reas_infr_df.rename({0: 'pre infr reas shen cpl', 1: 'post infr reas shen cpl'}, axis=1, inplace=True)

shen_mod_reas_infr_df = df_pivot.pivot(index='subject', columns='session', values='modularity')
shen_mod_reas_infr_df.rename({0: 'pre infr reas shen mod', 1: 'post infr reas shen mod'}, axis=1, inplace=True)

df_pivot = reas_infr_df[reas_infr_df['mask'] == 'craddock2012'].reset_index()
crad_eff_reas_infr_df = df_pivot.pivot(index='subject', columns='session', values='efficiency')
crad_eff_reas_infr_df.rename({0: 'pre infr reas crad eff', 1: 'post infr reas crad eff'}, axis=1, inplace=True)

crad_cpl_reas_infr_df = df_pivot.pivot(index='subject', columns='session', values='charpath')
crad_cpl_reas_infr_df.rename({0: 'pre infr reas crad cpl', 1: 'post infr reas crad cpl'}, axis=1, inplace=True)

crad_mod_reas_infr_df = df_pivot.pivot(index='subject', columns='session', values='modularity')
crad_mod_reas_infr_df.rename({0: 'pre infr reas crad mod', 1: 'post infr reas crad mod'}, axis=1, inplace=True)

#baseeral reasieval measures per session per parcellation
df_pivot = reas_base_df[reas_base_df['mask'] == 'shen2015'].reset_index()
shen_eff_reas_base_df = df_pivot.pivot(index='subject', columns='session', values='efficiency')
shen_eff_reas_base_df.rename({0: 'pre base reas shen eff', 1: 'post base reas shen eff'}, axis=1, inplace=True)

shen_cpl_reas_base_df = df_pivot.pivot(index='subject', columns='session', values='charpath')
shen_cpl_reas_base_df.rename({0: 'pre base reas shen cpl', 1: 'post base reas shen cpl'}, axis=1, inplace=True)

shen_mod_reas_base_df = df_pivot.pivot(index='subject', columns='session', values='modularity')
shen_mod_reas_base_df.rename({0: 'pre base reas shen mod', 1: 'post base reas shen mod'}, axis=1, inplace=True)

df_pivot = reas_base_df[reas_base_df['mask'] == 'craddock2012'].reset_index()
crad_eff_reas_base_df = df_pivot.pivot(index='subject', columns='session', values='efficiency')
crad_eff_reas_base_df.rename({0: 'pre base reas crad eff', 1: 'post base reas crad eff'}, axis=1, inplace=True)

crad_cpl_reas_base_df = df_pivot.pivot(index='subject', columns='session', values='charpath')
crad_cpl_reas_base_df.rename({0: 'pre base reas crad cpl', 1: 'post base reas crad cpl'}, axis=1, inplace=True)

crad_mod_reas_base_df = df_pivot.pivot(index='subject', columns='session', values='modularity')
crad_mod_reas_base_df.rename({0: 'pre base reas crad mod', 1: 'post base reas crad mod'}, axis=1, inplace=True)

In [ ]:
df_pivot = rsgt_df[rsgt_df['mask'] == 'shen2015'].reset_index()
shen_eff_df = df_pivot.pivot(index='subject', columns='session', values='efficiency')
shen_eff_df.rename({0: 'pre rest shen eff', 1: 'post rest shen eff'}, axis=1, inplace=True)

shen_cpl_df = df_pivot.pivot(index='subject', columns='session', values='charpath')
shen_cpl_df.rename({0: 'pre rest shen cpl', 1: 'post rest shen cpl'}, axis=1, inplace=True)

shen_mod_df = df_pivot.pivot(index='subject', columns='session', values='modularity')
shen_mod_df.rename({0: 'pre rest shen mod', 1: 'post rest shen mod'}, axis=1, inplace=True)

df_pivot = rsgt_df[rsgt_df['mask'] == 'craddock2012'].reset_index()
crad_eff_df = df_pivot.pivot(index='subject', columns='session', values='efficiency')
crad_eff_df.rename({0: 'pre rest crad eff', 1: 'post rest crad eff'}, axis=1, inplace=True)

crad_cpl_df = df_pivot.pivot(index='subject', columns='session', values='charpath')
crad_cpl_df.rename({0: 'pre rest crad cpl', 1: 'post rest crad cpl'}, axis=1, inplace=True)

crad_mod_df = df_pivot.pivot(index='subject', columns='session', values='modularity')
crad_mod_df.rename({0: 'pre rest crad mod', 1: 'post rest crad mod'}, axis=1, inplace=True)

In [ ]:
#demo_df[demo_df['Strt.Level'] < 0] = np.nan

In [ ]:
all_df = pd.concat((df, shen_eff_df, shen_cpl_df, shen_mod_df,
                    crad_eff_df, crad_cpl_df, crad_mod_df, 
                    head_size['normalized head size'],
                    shen_eff_retr_phys_df, shen_cpl_retr_phys_df, shen_mod_retr_phys_df,
                    crad_eff_retr_phys_df, crad_cpl_retr_phys_df, crad_mod_retr_phys_df, 
                    shen_eff_retr_genr_df, shen_cpl_retr_genr_df, shen_mod_retr_genr_df, 
                    crad_eff_retr_genr_df, crad_cpl_retr_genr_df, crad_mod_retr_genr_df,
                    shen_eff_phys_fci_df, shen_cpl_phys_fci_df, shen_mod_phys_fci_df,
                    crad_eff_phys_fci_df, crad_cpl_phys_fci_df, crad_mod_phys_fci_df,
                    shen_eff_ctrl_fci_df, shen_cpl_ctrl_fci_df, shen_mod_ctrl_fci_df,
                    crad_eff_ctrl_fci_df, crad_cpl_ctrl_fci_df, crad_mod_ctrl_fci_df, 
                    retr_phys_fd, retr_genr_fd, fci_phys_fd, fci_ctrl_fd, rs_fd), axis=1)

missing_df = pd.DataFrame(columns=all_df.keys(), index=all_df.index)

for key in missing_df.keys():
    missing_df[key] = all_df[key].isnull()

missing_df.replace({False: 0, True:1}, inplace=True)


In [ ]:
index = pd.MultiIndex.from_product([missing_df.keys(), all_df.keys()])
mnar = pd.DataFrame(columns=['corr', 'pval'], index=index)

In [ ]:
all_df.drop(['Sex', 'Class.Type'], axis=1, inplace=True)
all_df.dropna(inplace=True)

In [ ]:
small = all_df[['post phys fci shen cpl', 'post ctrl fci shen cpl', 'FSIQ2', 
                          'Mod', 'F', 'Age', 'normalized head size', 'post phys fci fd', 
                          'post ctrl fci fd','Strt.Level']].dropna()
small.rename({'post phys fci shen cpl': 'post_phys_cpl', 
                   'post ctrl fci shen cpl': 'post_ctrl_cpl', 
                   'FSIQ2': 'post_fsiq',
                   'Age': 'age_scan', 
                   'Mod': 'modeling',
                   'F': 'female',
                   'normalized head size': 'head_size', 
                   'post phys fci fd': 'post_phys_fd',
                   'post ctrl fci fd': 'post_ctrl_fd',
                   'Strt.Level': 'start_level'}, axis=1, inplace=True)
#small.head()
small.to_csv('/Users/Katie/Dropbox/Training/NBCLab19-Visualization/data/small_data_imp.csv', sep=',')

In [ ]:
all_df['delta rest shen mod'] = all_df['post rest shen mod'] - all_df['pre rest shen mod']
all_df['delta rest shen cpl'] = all_df['post rest shen cpl'] - all_df['pre rest shen cpl']
all_df['delta rest shen eff'] = all_df['post rest shen eff'] - all_df['pre rest shen eff']
all_df['delta rest crad mod'] = all_df['post rest crad mod'] - all_df['pre rest crad mod']
all_df['delta rest crad cpl'] = all_df['post rest crad cpl'] - all_df['pre rest crad cpl']
all_df['delta rest crad eff'] = all_df['post rest crad eff'] - all_df['pre rest crad eff']

all_df['delta phys retr shen eff'] = all_df['post phys retr shen eff'] - all_df['pre phys retr shen eff']
all_df['delta phys retr shen mod'] = all_df['post phys retr shen mod'] - all_df['pre phys retr shen mod']
all_df['delta phys retr shen cpl'] = all_df['post phys retr shen cpl'] - all_df['pre phys retr shen cpl']
all_df['delta phys retr crad eff'] = all_df['post phys retr crad eff'] - all_df['pre phys retr crad eff']
all_df['delta phys retr crad mod'] = all_df['post phys retr crad mod'] - all_df['pre phys retr crad mod']
all_df['delta phys retr crad cpl'] = all_df['post phys retr crad cpl'] - all_df['pre phys retr crad cpl']

all_df['delta gen retr shen eff'] = all_df['post gen retr shen eff'] - all_df['pre gen retr shen eff']
all_df['delta gen retr shen mod'] = all_df['post gen retr shen mod'] - all_df['pre gen retr shen mod']
all_df['delta gen retr shen cpl'] = all_df['post gen retr shen cpl'] - all_df['pre gen retr shen cpl']
all_df['delta gen retr crad eff'] = all_df['post gen retr crad eff'] - all_df['pre gen retr crad eff']
all_df['delta gen retr crad mod'] = all_df['post gen retr crad mod'] - all_df['pre gen retr crad mod']
all_df['delta gen retr crad cpl'] = all_df['post gen retr crad cpl'] - all_df['pre gen retr crad cpl']

all_df['delta phys fci shen eff'] = all_df['post phys fci shen eff'] - all_df['pre phys fci shen eff']
all_df['delta phys fci shen mod'] = all_df['post phys fci shen mod'] - all_df['pre phys fci shen mod']
all_df['delta phys fci shen cpl'] = all_df['post phys fci shen cpl'] - all_df['pre phys fci shen cpl']
all_df['delta phys fci crad eff'] = all_df['post phys fci crad eff'] - all_df['pre phys fci crad eff']
all_df['delta phys fci crad mod'] = all_df['post phys fci crad mod'] - all_df['pre phys fci crad mod']
all_df['delta phys fci crad cpl'] = all_df['post phys fci crad cpl'] - all_df['pre phys fci crad cpl']

all_df['delta ctrl fci shen eff'] = all_df['post ctrl fci shen eff'] - all_df['pre ctrl fci shen eff']
all_df['delta ctrl fci shen mod'] = all_df['post ctrl fci shen mod'] - all_df['pre ctrl fci shen mod']
all_df['delta ctrl fci shen cpl'] = all_df['post ctrl fci shen cpl'] - all_df['pre ctrl fci shen cpl']
all_df['delta ctrl fci crad eff'] = all_df['post ctrl fci crad eff'] - all_df['pre ctrl fci crad eff']
all_df['delta ctrl fci crad mod'] = all_df['post ctrl fci crad mod'] - all_df['pre ctrl fci crad mod']
all_df['delta ctrl fci crad cpl'] = all_df['post ctrl fci crad cpl'] - all_df['pre ctrl fci crad cpl']

all_df['delta ctrl fci fd'] = (all_df['post ctrl fci fd'] + all_df['pre ctrl fci fd']) / 2
all_df['delta phys fci fd'] = (all_df['post phys fci fd'] + all_df['pre phys fci fd']) / 2
all_df['delta phys retr fd'] = (all_df['post phys retr fd'] + all_df['pre phys retr fd']) / 2
all_df['delta gen retr fd'] = (all_df['post gen retr fd'] + all_df['pre gen retr fd']) / 2
all_df['delta rest fd'] = (all_df['post rest fd'] + all_df['pre rest fd']) / 2

In [ ]:
all_df_f = all_df[all_df['F'] == 1]
all_df_m = all_df[all_df['F'] == 0]

In [ ]:
sexes = ['Male', 'Female', 'All']
sessions = ['pre', 'post', 'delta']

# For all students, all together, how is topology related to IQ?

In [ ]:
#first things first, setting up our covariates
covariates = ['Age', 'Mod', 'Strt.Level', 'normalized head size', 'SexXClass', 'F']

In [ ]:
iqs = ['VCI', 'WMI', 'PRI', 'PSI', 'FSIQ']

In [ ]:
outcomes1 = ['pre phys fci shen eff', 'post phys fci shen eff', 'pre phys fci shen cpl', 
            'post phys fci shen cpl', 'pre phys fci shen mod', 'post phys fci shen mod',
            'pre ctrl fci shen eff', 'post ctrl fci shen eff', 'pre ctrl fci shen cpl', 
            'post ctrl fci shen cpl', 'pre ctrl fci shen mod', 'post ctrl fci shen mod', 
            'delta phys fci shen mod', 'delta phys fci shen cpl', 'delta phys fci shen eff',
            'delta ctrl fci shen mod', 'delta ctrl fci shen cpl', 'delta ctrl fci shen eff']
outcomes2 = ['pre rest shen eff', 'post rest shen eff', 'pre rest shen cpl', 
            'post rest shen cpl', 'pre rest shen mod', 'post rest shen mod', 
            'delta rest shen mod', 'delta rest shen cpl', 'delta rest shen eff']
outcomes3 = ['pre phys retr shen eff', 'post phys retr shen eff', 'pre phys retr shen cpl', 
            'post phys retr shen cpl', 'pre phys retr shen mod', 'post phys retr shen mod',
            'pre gen retr shen eff', 'post gen retr shen eff', 'pre gen retr shen cpl', 
            'post gen retr shen cpl', 'pre gen retr shen mod', 'post gen retr shen mod', 
            'delta phys retr shen mod', 'delta phys retr shen cpl', 'delta phys retr shen eff',
            'delta gen retr shen mod', 'delta gen retr shen cpl', 'delta gen retr shen eff']

all_out = outcomes1 + outcomes2 + outcomes3

all_mc_alpha,_ = liji_sidak_mc(all_df[all_out], 0.05)
print('Shen', all_mc_alpha)

In [ ]:
outcomes1 = ['pre phys fci crad eff', 'post phys fci crad eff', 'pre phys fci crad cpl', 
            'post phys fci crad cpl', 'pre phys fci crad mod', 'post phys fci crad mod',
            'pre ctrl fci crad eff', 'post ctrl fci crad eff', 'pre ctrl fci crad cpl', 
            'post ctrl fci crad cpl', 'pre ctrl fci crad mod', 'post ctrl fci crad mod', 
            'delta phys fci crad mod', 'delta phys fci crad cpl', 'delta phys fci crad eff',
            'delta ctrl fci crad mod', 'delta ctrl fci crad cpl', 'delta ctrl fci crad eff']
outcomes2 = ['pre rest crad eff', 'post rest crad eff', 'pre rest crad cpl', 
            'post rest crad cpl', 'pre rest crad mod', 'post rest crad mod', 
            'delta rest crad mod', 'delta rest crad cpl', 'delta rest crad eff']
outcomes3 = ['pre phys retr crad eff', 'post phys retr crad eff', 'pre phys retr crad cpl', 
            'post phys retr crad cpl', 'pre phys retr crad mod', 'post phys retr crad mod',
            'pre gen retr crad eff', 'post gen retr crad eff', 'pre gen retr crad cpl', 
            'post gen retr crad cpl', 'pre gen retr crad mod', 'post gen retr crad mod', 
            'delta phys retr crad mod', 'delta phys retr crad cpl', 'delta phys retr crad eff',
            'delta gen retr crad mod', 'delta gen retr crad cpl', 'delta gen retr crad eff']

all_out = outcomes1 + outcomes2 + outcomes3

#
all_mc_alpha,_ = liji_sidak_mc(all_df[all_out], 0.05)
print('Craddock', all_mc_alpha)

## Rest and brain organization!

In [ ]:
outcomes = ['pre rest shen eff', 'post rest shen eff', 'pre rest shen cpl', 
            'post rest shen cpl', 'pre rest shen mod', 'post rest shen mod', 
            'delta rest shen mod', 'delta rest shen cpl', 'delta rest shen eff']

In [ ]:
outcomes = ['pre rest crad eff', 'post rest crad eff', 'pre rest crad cpl', 
            'post rest crad cpl', 'pre rest crad mod', 'post rest crad mod', 
            'delta rest crad mod', 'delta rest crad cpl', 'delta rest crad eff']

In [ ]:
mc_alpha,_ = liji_sidak_mc(all_df[outcomes], 0.05)

In [ ]:
index = pd.MultiIndex.from_product((sessions, iqs))
brainXIQ_df = pd.DataFrame(index=index, columns=['F-statistic', 'Prob (F-statistic)', 
                                                     'AIC', 'BIC', 'IQ_pval', 'IQXClass_pval'])

all_outcome_mc,_ = liji_sidak_mc(all_df[outcomes], 0.05)
for i in np.arange(0,len(outcomes)):
    for iq_meas in iqs:
        y = all_df[outcomes[i]]
        x = all_df[['{0}1'.format(iq_meas), 
                    '{0}1XSex'.format(iq_meas), 
                    '{0}1XClass'.format(iq_meas),
                    '{0}1XClassXSex'.format(iq_meas), 
                    '{0} fd'.format(outcomes[i][:-9])] + covariates]

        x = sm.add_constant(x)
        reg1 = sm.OLS(endog=y, 
                      exog=x, 
                      missing='drop')
        if len(x.dropna()) == len(y.dropna()):
            linreg = Lasso()
            linreg.fit(x.dropna(),y.dropna())
        brain_on_pre_iq = reg1.fit()
        brainXIQ_df.at[('pre', iq_meas), 'F-statistic'] = brain_on_pre_iq.fvalue
        brainXIQ_df.at[('pre', iq_meas), 'Prob (F-statistic)'] = brain_on_pre_iq.f_pvalue
        brainXIQ_df.at[('pre', iq_meas), 'AIC'] = brain_on_pre_iq.aic
        brainXIQ_df.at[('pre', iq_meas), 'BIC'] = brain_on_pre_iq.bic
        brainXIQ_df.at[('pre', iq_meas), '{0}_IQ_pval'.format(iq_meas)] = brain_on_pre_iq.pvalues[1]
        brainXIQ_df.at[('pre', iq_meas), '{0}_IQXClass_pval'.format(iq_meas)] = brain_on_pre_iq.pvalues[2]
        if brain_on_pre_iq.f_pvalue < all_mc_alpha:
            print(brain_on_pre_iq.summary())
        #print('Course grade regressed on pre IQ, female students\n', grade_on_iq.summary())
        x = all_df[['{0}2'.format(iq_meas), 
                    '{0}2XSex'.format(iq_meas), 
                    '{0}2XClass'.format(iq_meas),
                    '{0}2XClassXSex'.format(iq_meas), 
                    '{0} fd'.format(outcomes[i][:-9])] + covariates]

        x = sm.add_constant(x)
        reg1 = sm.OLS(endog=y, 
                      exog=x, 
                      missing='drop')
        brain_on_post_iq = reg1.fit()
        brainXIQ_df.at[('post', iq_meas), 'F-statistic'] = brain_on_post_iq.fvalue
        brainXIQ_df.at[('post', iq_meas), 'Prob (F-statistic)'] = brain_on_post_iq.f_pvalue
        brainXIQ_df.at[('post', iq_meas), 'AIC'] = brain_on_post_iq.aic
        brainXIQ_df.at[('post', iq_meas), 'BIC'] = brain_on_post_iq.bic
        brainXIQ_df.at[('post', iq_meas), '{0}_IQ_pval'.format(outcomes[i])] = brain_on_post_iq.pvalues[1]
        brainXIQ_df.at[('post', iq_meas), '{0}_IQXClass_pval'.format(outcomes[i])] = brain_on_post_iq.pvalues[2]
        if brain_on_post_iq.f_pvalue < all_mc_alpha:
            print(brain_on_post_iq.summary())
        #print('Course grade regressed on post IQ, female students\n', grade_on_iq.summary())
        x = all_df[['delta{0}'.format(iq_meas), 
                    'delta{0}XSex'.format(iq_meas), 
                    'delta{0}XClass'.format(iq_meas), 
                    'delta{0}XClassXSex'.format(iq_meas), 
                    '{0} fd'.format(outcomes[i][:-9])] + covariates]

        x = sm.add_constant(x)
        reg1 = sm.OLS(endog=y, 
                      exog=x, 
                      missing='drop')
        brain_on_delta_iq = reg1.fit()
        brainXIQ_df.at[('delta', iq_meas), 'F-statistic'] = brain_on_delta_iq.fvalue
        brainXIQ_df.at[('delta', iq_meas), 'Prob (F-statistic)'] = brain_on_delta_iq.f_pvalue
        brainXIQ_df.at[('delta', iq_meas), 'AIC'] = brain_on_delta_iq.aic
        brainXIQ_df.at[('delta', iq_meas), 'BIC'] = brain_on_delta_iq.bic
        brainXIQ_df.at[('delta', iq_meas), '{0}_IQ_pval'.format(outcomes[i])] = brain_on_delta_iq.pvalues[1]
        brainXIQ_df.at[('delta', iq_meas), '{0}_IQXClass_pval'.format(outcomes[i])] = brain_on_delta_iq.pvalues[2]
        if brain_on_delta_iq.f_pvalue < all_mc_alpha:
            print(brain_on_delta_iq.summary())
    brainXIQ_df.to_csv(join(data_dir, 'rescored', 'iqX{0}_all.csv'.format(outcomes[i])))

In [ ]:
if len(x.dropna()) == len(y.dropna()):
    linreg = Lasso(alpha=0.5)
    q = linreg.fit(x.dropna(),y.dropna())

In [ ]:
q.score(x.dropna(),y.dropna())

## Retrieval task brain organization and IQ!

In [ ]:
outcomes = ['pre phys retr crad eff', 'post phys retr crad eff', 'pre phys retr crad cpl', 
            'post phys retr crad cpl', 'pre phys retr crad mod', 'post phys retr crad mod',
            'pre gen retr crad eff', 'post gen retr crad eff', 'pre gen retr crad cpl', 
            'post gen retr crad cpl', 'pre gen retr crad mod', 'post gen retr crad mod', 
            'delta phys retr crad mod', 'delta phys retr crad cpl', 'delta phys retr crad eff',
            'delta gen retr crad mod', 'delta gen retr crad cpl', 'delta gen retr crad eff']

In [ ]:
outcomes = ['pre phys retr shen eff', 'post phys retr shen eff', 'pre phys retr shen cpl', 
            'post phys retr shen cpl', 'pre phys retr shen mod', 'post phys retr shen mod',
            'pre gen retr shen eff', 'post gen retr shen eff', 'pre gen retr shen cpl', 
            'post gen retr shen cpl', 'pre gen retr shen mod', 'post gen retr shen mod', 
            'delta phys retr shen mod', 'delta phys retr shen cpl', 'delta phys retr shen eff',
            'delta gen retr shen mod', 'delta gen retr shen cpl', 'delta gen retr shen eff']

In [ ]:
index = pd.MultiIndex.from_product((sessions, iqs))
brainXIQ_df = pd.DataFrame(index=index, columns=['F-statistic', 'Prob (F-statistic)', 
                                                     'AIC', 'BIC', 'IQ_pval', 'IQXClass_pval'])
all_outcome_mc,_ = liji_sidak_mc(all_df[outcomes], 0.05)
for i in np.arange(0,len(outcomes)):
    for iq_meas in iqs:
        y = all_df[outcomes[i]]
        if outcomes[0][:-9][-4:] != 'rest':
            x = all_df[['{0}1'.format(iq_meas), 
                        #'{0}1XSex'.format(iq_meas), 
                        '{0}1XClass'.format(iq_meas),
                        #'{0}1XClassXSex'.format(iq_meas), 
                        '{0} fd'.format(outcomes[i][:-9])] + covariates]
        else:
            x = all_df[['{0}1'.format(iq_meas), 
                        '{0}1XSex'.format(iq_meas), 
                        '{0}1XClass'.format(iq_meas),
                        '{0}1XClassXSex'.format(iq_meas)] + covariates]
            
        x = sm.add_constant(x)
        reg1 = sm.OLS(endog=y, 
                      exog=x, 
                      missing='drop')
        brain_on_pre_iq = reg1.fit()
        brainXIQ_df.at[('pre', iq_meas), 'F-statistic'] = brain_on_pre_iq.fvalue
        brainXIQ_df.at[('pre', iq_meas), 'Prob (F-statistic)'] = brain_on_pre_iq.f_pvalue
        brainXIQ_df.at[('pre', iq_meas), 'AIC'] = brain_on_pre_iq.aic
        brainXIQ_df.at[('pre', iq_meas), 'BIC'] = brain_on_pre_iq.bic
        brainXIQ_df.at[('pre', iq_meas), '{0}_IQ_pval'.format(iq_meas)] = brain_on_pre_iq.pvalues[1]
        brainXIQ_df.at[('pre', iq_meas), '{0}_IQXClass_pval'.format(iq_meas)] = brain_on_pre_iq.pvalues[2]
        if brain_on_pre_iq.f_pvalue < all_mc_alpha and brain_on_pre_iq.pvalues['{0}1'.format(iq_meas)] < 0.05:
            print(brain_on_pre_iq.summary())
        #print('Course grade regressed on pre IQ, female students\n', grade_on_iq.summary())
        if outcomes[0][:-9][-4:] != 'rest':
            x = all_df[['{0}2'.format(iq_meas), 
                        #'{0}2XSex'.format(iq_meas), 
                        '{0}2XClass'.format(iq_meas),
                        #'{0}2XClassXSex'.format(iq_meas), 
                        '{0} fd'.format(outcomes[i][:-9])] + covariates]
        else:
            x = all_df[['{0}2'.format(iq_meas), 
                        '{0}2XSex'.format(iq_meas), 
                        '{0}2XClass'.format(iq_meas),
                        '{0}2XClassXSex'.format(iq_meas)] + covariates]
        x = sm.add_constant(x)
        reg1 = sm.OLS(endog=y, 
                      exog=x, 
                      missing='drop')
        brain_on_post_iq = reg1.fit()
        brainXIQ_df.at[('post', iq_meas), 'F-statistic'] = brain_on_post_iq.fvalue
        brainXIQ_df.at[('post', iq_meas), 'Prob (F-statistic)'] = brain_on_post_iq.f_pvalue
        brainXIQ_df.at[('post', iq_meas), 'AIC'] = brain_on_post_iq.aic
        brainXIQ_df.at[('post', iq_meas), 'BIC'] = brain_on_post_iq.bic
        brainXIQ_df.at[('post', iq_meas), '{0}_IQ_pval'.format(outcomes[i])] = brain_on_post_iq.pvalues[1]
        brainXIQ_df.at[('post', iq_meas), '{0}_IQXClass_pval'.format(outcomes[i])] = brain_on_post_iq.pvalues[2]
        if brain_on_post_iq.f_pvalue < all_mc_alpha and brain_on_post_iq.pvalues['{0}2'.format(iq_meas)] < 0.05:
            print(brain_on_post_iq.summary())
        #print('Course grade regressed on post IQ, female students\n', grade_on_iq.summary())
        if outcomes[0][:-9][-4:] != 'rest':
            x = all_df[['delta{0}'.format(iq_meas), 
                        #'delta{0}XSex'.format(iq_meas), 
                        'delta{0}XClass'.format(iq_meas), 
                        #'delta{0}XClassXSex'.format(iq_meas), 
                        '{0} fd'.format(outcomes[i][:-9])] + covariates]
        else:
            x = all_df[['delta{0}'.format(iq_meas), 
                        'delta{0}XSex'.format(iq_meas), 
                        'delta{0}XClass'.format(iq_meas),
                        'delta{0}XClassXSex'.format(iq_meas)] + covariates]
        x = sm.add_constant(x)
        reg1 = sm.OLS(endog=y, 
                      exog=x, 
                      missing='drop')
        brain_on_delta_iq = reg1.fit()
        brainXIQ_df.at[('delta', iq_meas), 'F-statistic'] = brain_on_delta_iq.fvalue
        brainXIQ_df.at[('delta', iq_meas), 'Prob (F-statistic)'] = brain_on_delta_iq.f_pvalue
        brainXIQ_df.at[('delta', iq_meas), 'AIC'] = brain_on_delta_iq.aic
        brainXIQ_df.at[('delta', iq_meas), 'BIC'] = brain_on_delta_iq.bic
        brainXIQ_df.at[('delta', iq_meas), '{0}_IQ_pval'.format(outcomes[i])] = brain_on_delta_iq.pvalues[1]
        brainXIQ_df.at[('delta', iq_meas), '{0}_IQXClass_pval'.format(outcomes[i])] = brain_on_delta_iq.pvalues[2]
        if brain_on_delta_iq.f_pvalue < all_mc_alpha and brain_on_delta_iq.pvalues['delta{0}'.format(iq_meas)] < 0.05:
            print(brain_on_delta_iq.summary())
    brainXIQ_df.to_csv(join(data_dir, 'rescored', 'iqX{0}_all.csv'.format(outcomes[i])))

## Reasoning task and IQ!

In [ ]:
outcomes = ['pre infr reas shen eff', 'post infr reas shen eff', 'pre infr reas shen cpl', 
            'post infr reas shen cpl', 'pre infr reas shen mod', 'post infr reas shen mod',
            'pre base reas shen eff', 'post base reas shen eff', 'pre base reas shen cpl', 
            'post base reas shen cpl', 'pre base reas shen mod', 'post base reas shen mod',
            'delta infr reas shen mod', 'delta infr reas shen cpl', 'delta infr reas shen eff',
            'delta base reas shen mod', 'delta base reas shen cpl', 'delta base reas shen eff']

In [ ]:
outcomes = ['pre infr reas crad eff', 'post infr reas crad eff', 'pre infr reas crad cpl', 
            'post infr reas crad cpl', 'pre infr reas crad mod', 'post infr reas crad mod',
            'pre base reas crad eff', 'post base reas crad eff', 'pre base reas crad cpl', 
            'post base reas crad cpl', 'pre base reas crad mod', 'post base reas crad mod',
            'delta infr reas crad mod', 'delta infr reas crad cpl', 'delta infr reas crad eff',
            'delta base reas crad mod', 'delta base reas crad cpl', 'delta base reas crad eff']

In [ ]:
index = pd.MultiIndex.from_product((sessions, iqs))
brainXIQ_df = pd.DataFrame(index=index, columns=['F-statistic', 'Prob (F-statistic)', 
                                                     'AIC', 'BIC', 'IQ_pval', 'IQXClass_pval'])
all_outcome_mc,_ = liji_sidak_mc(all_df[outcomes], 0.05)
for i in np.arange(0,len(outcomes)):
    for iq_meas in iqs:
        y = all_df[outcomes[i]]
        if outcomes[0][:-9][-4:] != 'rest':
            x = all_df[['{0}1'.format(iq_meas), 
                        '{0}1XSex'.format(iq_meas), 
                        '{0}1XClass'.format(iq_meas),
                        '{0}1XClassXSex'.format(iq_meas), 
                        '{0} fd'.format(outcomes[i][:-9])] + covariates]
        else:
            x = all_df[['{0}1'.format(iq_meas), 
                        '{0}1XSex'.format(iq_meas), 
                        '{0}1XClass'.format(iq_meas),
                        '{0}1XClassXSex'.format(iq_meas)] + covariates]
            
        x = sm.add_constant(x)
        reg1 = sm.OLS(endog=y, 
                      exog=x, 
                      missing='drop')
        brain_on_pre_iq = reg1.fit()
        brainXIQ_df.at[('pre', iq_meas), 'F-statistic'] = brain_on_pre_iq.fvalue
        brainXIQ_df.at[('pre', iq_meas), 'Prob (F-statistic)'] = brain_on_pre_iq.f_pvalue
        brainXIQ_df.at[('pre', iq_meas), 'AIC'] = brain_on_pre_iq.aic
        brainXIQ_df.at[('pre', iq_meas), 'BIC'] = brain_on_pre_iq.bic
        brainXIQ_df.at[('pre', iq_meas), '{0}_IQ_pval'.format(iq_meas)] = brain_on_pre_iq.pvalues[1]
        brainXIQ_df.at[('pre', iq_meas), '{0}_IQXClass_pval'.format(iq_meas)] = brain_on_pre_iq.pvalues[2]
        if brain_on_pre_iq.f_pvalue < all_mc_alpha and brain_on_pre_iq.pvalues['{0}1'.format(iq_meas)] < 0.05:
            print(brain_on_pre_iq.summary())
        #print('Course grade regressed on pre IQ, female students\n', grade_on_iq.summary())
        if outcomes[0][:-9][-4:] != 'rest':
            x = all_df[['{0}2'.format(iq_meas), 
                        '{0}2XSex'.format(iq_meas), 
                        '{0}2XClass'.format(iq_meas),
                        '{0}2XClassXSex'.format(iq_meas), 
                        '{0} fd'.format(outcomes[i][:-9])] + covariates]
        else:
            x = all_df[['{0}2'.format(iq_meas), 
                        '{0}2XSex'.format(iq_meas), 
                        '{0}2XClass'.format(iq_meas),
                        '{0}2XClassXSex'.format(iq_meas)] + covariates]
        x = sm.add_constant(x)
        reg1 = sm.OLS(endog=y, 
                      exog=x, 
                      missing='drop')
        brain_on_post_iq = reg1.fit()
        brainXIQ_df.at[('post', iq_meas), 'F-statistic'] = brain_on_post_iq.fvalue
        brainXIQ_df.at[('post', iq_meas), 'Prob (F-statistic)'] = brain_on_post_iq.f_pvalue
        brainXIQ_df.at[('post', iq_meas), 'AIC'] = brain_on_post_iq.aic
        brainXIQ_df.at[('post', iq_meas), 'BIC'] = brain_on_post_iq.bic
        brainXIQ_df.at[('post', iq_meas), '{0}_IQ_pval'.format(outcomes[i])] = brain_on_post_iq.pvalues[1]
        brainXIQ_df.at[('post', iq_meas), '{0}_IQXClass_pval'.format(outcomes[i])] = brain_on_post_iq.pvalues[2]
        if brain_on_post_iq.f_pvalue < all_mc_alpha and brain_on_post_iq.pvalues['{0}2'.format(iq_meas)] < 0.05:
            print(brain_on_post_iq.summary())
        #print('Course grade regressed on post IQ, female students\n', grade_on_iq.summary())

        if outcomes[0][:-9][-4:] != 'rest':
            x = all_df[['delta{0}'.format(iq_meas), 
                        'delta{0}XSex'.format(iq_meas), 
                        'delta{0}XClass'.format(iq_meas), 
                        'delta{0}XClassXSex'.format(iq_meas), 
                        '{0} fd'.format(outcomes[i][:-9])] + covariates]
        else:
            x = all_df[['delta{0}'.format(iq_meas), 
                        'delta{0}XSex'.format(iq_meas), 
                        'delta{0}XClass'.format(iq_meas),
                        'delta{0}XClassXSex'.format(iq_meas)] + covariates]
        x = sm.add_constant(x)
        reg1 = sm.OLS(endog=y, 
                      exog=x, 
                      missing='drop')
        brain_on_delta_iq = reg1.fit()
        brainXIQ_df.at[('delta', iq_meas), 'F-statistic'] = brain_on_delta_iq.fvalue
        brainXIQ_df.at[('delta', iq_meas), 'Prob (F-statistic)'] = brain_on_delta_iq.f_pvalue
        brainXIQ_df.at[('delta', iq_meas), 'AIC'] = brain_on_delta_iq.aic
        brainXIQ_df.at[('delta', iq_meas), 'BIC'] = brain_on_delta_iq.bic
        brainXIQ_df.at[('delta', iq_meas), '{0}_IQ_pval'.format(outcomes[i])] = brain_on_delta_iq.pvalues[1]
        brainXIQ_df.at[('delta', iq_meas), '{0}_IQXClass_pval'.format(outcomes[i])] = brain_on_delta_iq.pvalues[2]
        if brain_on_delta_iq.f_pvalue < all_mc_alpha and brain_on_delta_iq.pvalues['delta{0}'.format(iq_meas)] < 0.05:
            print(brain_on_delta_iq.summary())
    brainXIQ_df.to_csv(join(data_dir, 'rescored', 'iqX{0}_all.csv'.format(outcome)))

## FCI Task

In [ ]:
outcomes = ['pre phys fci crad eff', 'post phys fci crad eff', 'pre phys fci crad cpl', 
            'post phys fci crad cpl', 'pre phys fci crad mod', 'post phys fci crad mod',
            'pre ctrl fci crad eff', 'post ctrl fci crad eff', 'pre ctrl fci crad cpl', 
            'post ctrl fci crad cpl', 'pre ctrl fci crad mod', 'post ctrl fci crad mod', 
            'delta phys fci crad mod', 'delta phys fci crad cpl', 'delta phys fci crad eff',
            'delta ctrl fci crad mod', 'delta ctrl fci crad cpl', 'delta ctrl fci crad eff']

In [ ]:
outcomes = ['pre phys fci shen eff', 'post phys fci shen eff', 'pre phys fci shen cpl', 
            'post phys fci shen cpl', 'pre phys fci shen mod', 'post phys fci shen mod',
            'delta phys fci shen mod', 'delta phys fci shen cpl', 'delta phys fci shen eff']

In [ ]:
outcomes = ['pre phys fci shen eff', 'post phys fci shen eff', 'pre phys fci shen cpl', 
            'post phys fci shen cpl', 'pre phys fci shen mod', 'post phys fci shen mod',
            'pre ctrl fci shen eff', 'post ctrl fci shen eff', 'pre ctrl fci shen cpl', 
            'post ctrl fci shen cpl', 'pre ctrl fci shen mod', 'post ctrl fci shen mod', 
            'delta phys fci shen mod', 'delta phys fci shen cpl', 'delta phys fci shen eff',
            'delta ctrl fci shen mod', 'delta ctrl fci shen cpl', 'delta ctrl fci shen eff']

In [ ]:
index = pd.MultiIndex.from_product((sessions, iqs))
brainXIQ_df = pd.DataFrame(index=index, columns=['F-statistic', 'Prob (F-statistic)', 
                                                     'AIC', 'BIC', 'IQ_pval', 'IQXClass_pval'])
all_outcome_mc,_ = liji_sidak_mc(all_df[outcomes], 0.05)
for i in np.arange(0,len(outcomes)):
    for iq_meas in iqs:
        y = all_df[outcomes[i]]
        if outcomes[0][:-9][-4:] != 'rest':
            x = all_df[['{0}1'.format(iq_meas), '{0} fd'.format(outcomes[i][:-9])] + covariates]
                        #'{0}1XSex'.format(iq_meas), 
                        #'{0}1XClass'.format(iq_meas),
                        #'{0}1XClassXSex'.format(iq_meas), 
                        
        else:
            x = all_df[['{0}1'.format(iq_meas)] 
                        #'{0}1XSex'.format(iq_meas), 
                        #'{0}1XClass'.format(iq_meas),
                        #'{0}1XClassXSex'.format(iq_meas)] 
                        + covariates]
            
        x = sm.add_constant(x)
        reg1 = sm.OLS(endog=y, 
                      exog=x, 
                      missing='drop')
        brain_on_pre_iq = reg1.fit()
        brainXIQ_df.at[('pre', iq_meas), 'F-statistic'] = brain_on_pre_iq.fvalue
        brainXIQ_df.at[('pre', iq_meas), 'Prob (F-statistic)'] = brain_on_pre_iq.f_pvalue
        brainXIQ_df.at[('pre', iq_meas), 'AIC'] = brain_on_pre_iq.aic
        brainXIQ_df.at[('pre', iq_meas), 'BIC'] = brain_on_pre_iq.bic
        brainXIQ_df.at[('pre', iq_meas), '{0}_IQ_pval'.format(iq_meas)] = brain_on_pre_iq.pvalues[1]
        brainXIQ_df.at[('pre', iq_meas), '{0}_IQXClass_pval'.format(iq_meas)] = brain_on_pre_iq.pvalues[2]
        if brain_on_pre_iq.f_pvalue < all_mc_alpha and brain_on_pre_iq.pvalues['{0}1'.format(iq_meas)] < 0.05:
            print(brain_on_pre_iq.summary())
        #print('Course grade regressed on pre IQ, female students\n', grade_on_iq.summary())
        if outcomes[0][:-9][-4:] != 'rest':
            x = all_df[['{0}2'.format(iq_meas), 
                        '{0}2XSex'.format(iq_meas), 
                        '{0}2XClass'.format(iq_meas),
                        '{0}2XClassXSex'.format(iq_meas), 
                        '{0} fd'.format(outcomes[i][:-9])] + covariates]
        else:
            x = all_df[['{0}2'.format(iq_meas), 
                        '{0}2XSex'.format(iq_meas), 
                        '{0}2XClass'.format(iq_meas),
                        '{0}2XClassXSex'.format(iq_meas)] + covariates]
        x = sm.add_constant(x)
        reg1 = sm.OLS(endog=y, 
                      exog=x, 
                      missing='drop')
        brain_on_post_iq = reg1.fit()
        brainXIQ_df.at[('post', iq_meas), 'F-statistic'] = brain_on_post_iq.fvalue
        brainXIQ_df.at[('post', iq_meas), 'Prob (F-statistic)'] = brain_on_post_iq.f_pvalue
        brainXIQ_df.at[('post', iq_meas), 'AIC'] = brain_on_post_iq.aic
        brainXIQ_df.at[('post', iq_meas), 'BIC'] = brain_on_post_iq.bic
        brainXIQ_df.at[('post', iq_meas), '{0}_IQ_pval'.format(outcomes[i])] = brain_on_post_iq.pvalues[1]
        brainXIQ_df.at[('post', iq_meas), '{0}_IQXClass_pval'.format(outcomes[i])] = brain_on_post_iq.pvalues[2]
        if brain_on_post_iq.f_pvalue < all_mc_alpha and brain_on_post_iq.pvalues['{0}2'.format(iq_meas)] < 0.05:
            print(brain_on_post_iq.summary())
        #print('Course grade regressed on post IQ, female students\n', grade_on_iq.summary())
        
        if outcomes[0][:-9][-4:] != 'rest':
            x = all_df[['delta{0}'.format(iq_meas), 
                        #'delta{0}XSex'.format(iq_meas), 
                        #'delta{0}XClass'.format(iq_meas), 
                        #'delta{0}XClassXSex'.format(iq_meas), 
                        '{0} fd'.format(outcomes[i][:-9])] + covariates
                      ]
        else:
            x = all_df[['delta{0}'.format(iq_meas), 
                        #'delta{0}XSex'.format(iq_meas), 
                        #'delta{0}XClass'.format(iq_meas),
                        '#delta{0}XClassXSex'.format(iq_meas)] + covariates
                      ]
        x = sm.add_constant(x)
        reg1 = sm.OLS(endog=y, 
                      exog=x, 
                      missing='drop')
        brain_on_delta_iq = reg1.fit()
        brainXIQ_df.at[('delta', iq_meas), 'F-statistic'] = brain_on_delta_iq.fvalue
        brainXIQ_df.at[('delta', iq_meas), 'Prob (F-statistic)'] = brain_on_delta_iq.f_pvalue
        brainXIQ_df.at[('delta', iq_meas), 'AIC'] = brain_on_delta_iq.aic
        brainXIQ_df.at[('delta', iq_meas), 'BIC'] = brain_on_delta_iq.bic
        brainXIQ_df.at[('delta', iq_meas), '{0}_IQ_pval'.format(outcomes[i])] = brain_on_delta_iq.pvalues[1]
        brainXIQ_df.at[('delta', iq_meas), '{0}_IQXClass_pval'.format(outcomes[i])] = brain_on_delta_iq.pvalues[2]
        if brain_on_delta_iq.f_pvalue < all_mc_alpha and brain_on_delta_iq.pvalues['delta{0}'.format(iq_meas)] < 0.05:
            print(brain_on_delta_iq.summary())
    brainXIQ_df.to_csv(join(data_dir, 'rescored', 'iqX{0}_all.csv'.format(outcomes[i])))

# Significant Results
## Resting state
None.
## Retrieval
### Physics
Post Full Scale IQ & Post Charpath, Efficiency: IQ, Sex, Class, and all interactions.<br>
Post Perceptual Reasoning & Post Charpath: IQXSex, IQXSexXClass, SexXClass<br>
Delta Perceptual Reasoning & Post Charpath ($\alpha$ < 0.05): IQXSec, IQXSexXClass, SexXClass.
### General
Delta Perceptual Reasoning & Post Charpath, Efficiency: IQXSexXClass.

## Reasoning
### Transitive Inference
Delta Perceptual Reasoning & Pre Charpath, Efficiency: Class, SexXClass.<br>
Pre Working Memory & Delta Charpath, Efficiency: IQ, Sex, Class, and all interactions.<br>
### Baseline
Pre Verbal Comprehension & Pre Charpath, Efficiency (not all $\alpha$ < 0.01): IQ, Sex, Class, and all interactions.<br>
Pre Full Scale IQ & Pre Charpath, Efficiency (not all $\alpha$ < 0.01): IQ, IQxSex, Sex.<br>
Pre Full Scale IQ & Post Charpath, Efficiency ($\alpha$ < 0.05): IQXSex, IQXClass, IQXSexXClass, Sex, Class (eff only), SexXClass (chl only).<br>

## Force Concept Inventory
### FCI Questions
Post Full Scale IQ & Post Efficiency: IQ, Sex, Class, and all the interactions.<br>
Post Full Scale IQ & Post Charpath: IQXSex, IQXClass•, IQXSexXClass, Sex•, SexXClass.<br>
Delta Full Scale IQ & Post Efficiency: IQ, IQXSex, IQXClass•, IQXSexXClass.<br>
Post Pereptual Reasoning & Post Efficiency: IQXSex•, IQXSexXClass, Sex•, SexXClass•.<br>
Post Perceptual Reasoning & Post Charpath: IQXSex, IQXClass•, IQXSexXClass, Sex*, SexXClass.<br>
Delta Perceptual Reasoning & Post Charpath, Efficiency: IQ, IQXSex, IQXClass, IQXSexXClass.<br>
### Baseline Questions
Post Full Scale IQ & Post Charpath, Efficiency ($\alpha$ < 0.05): IQXSex, Sex.

# time for figures!

In [ ]:
crayons_l = sns.crayon_palette(['Vivid Tangerine', 'Cornflower'])
crayons_d = sns.crayon_palette(['Brick Red', 'Midnight Blue'])

f = sns.diverging_palette(8.2, 44, s=85, l=50, n=200)
m = sns.diverging_palette(243, 278, s=85, l=50, n=200)

f_2 = sns.crayon_palette(['Red Orange', 'Vivid Tangerine'])
m_2 = sns.crayon_palette(['Cerulean', 'Cornflower'])

# Physics Retrieval & PRIQ

In [ ]:
h = sns.lmplot('PRI2', 'post phys retr shen cpl', data=df_m, hue='Mod', 
           palette=m_2, legend_out=True, height=10., )

h.set_xlabels('Post-instruction perceptual reasoning')
h.set_ylabels('Post-instruction characteristic path length')
h.savefig(join(fig_dir, 'RETR_phys_post-cplXpost-priq_m.png'), dpi=300)

In [ ]:
h = sns.lmplot('PRI2', 'post phys retr shen cpl', data=df_f, hue='Mod', 
           palette=f_2, legend_out=True, height=10.)

h.set_xlabels('Post-instruction perceptual reasoning')
h.set_ylabels('Post-instruction characteristic path length')
h.savefig(join(fig_dir, 'RETR_phys_post-cplXpost-priq_f.png'), dpi=300)

# General retrieval and deltaPRIQ

In [ ]:
h = sns.lmplot('deltaPRI', 'post gen retr shen mod', data=df_m, hue='Mod', 
           palette=m_2, legend_out=True, height=10.)

h.set_xlabels('Change in perceptual reasoning')
h.set_ylabels('Post-instruction network modularity')
h.savefig(join(fig_dir, 'RETR_gen_post-modXdelta-priq_m.png'), dpi=300)

In [ ]:
h = sns.lmplot('deltaPRI', 'post gen retr shen mod', data=df_f, hue='Mod', 
           palette=f_2, legend_out=True, height=10.)

h.set_xlabels('Change in perceptual reasoning')
h.set_ylabels('Post-instruction network modularity')
h.savefig(join(fig_dir, 'RETR_gen_post-modXdelta-priq_f.png'), dpi=300)

# Physics Retrieval & FSIQ

In [ ]:
h = sns.lmplot('FSIQ2', 'post phys retr shen cpl', data=df_m, hue='Mod', 
           palette=m_2, legend_out=True, height=10.)

h.set_xlabels('Post-instruction full-scale IQ')
h.set_ylabels('Post-instruction network efficiency')
h.savefig(join(fig_dir, 'RETR_phys_post-cplXpost-fsiq_m.png'), dpi=300)

In [ ]:
h = sns.lmplot('FSIQ2', 'post phys retr shen cpl', data=df_m, hue='Mod', 
           palette=m_2, legend_out=True, height=10.)

h.set_xlabels('Post-instruction full-scale IQ')
h.set_ylabels('Post-instruction characteristic path length')
#h.savefig(join(fig_dir, 'pre_prXpre_eff_inf-m.png'), dpi=300)

In [ ]:
h = sns.lmplot('FSIQ2', 'post phys retr shen cpl', data=df_f, hue='Mod', 
           palette=f_2, legend_out=True, height=10.)

h.set_xlabels('Post-instruction full-scale IQ')
h.set_ylabels('Post-instruction network-efficiency')
h.savefig(join(fig_dir, 'RETR_phys_post-cplXpost-fsiq_f.png'), dpi=300)

In [ ]:
h = sns.lmplot('FSIQ2', 'post phys retr shen cpl', data=df_f, hue='Mod', 
           palette=f_2, legend_out=True, height=10.)

h.set_xlabels('Post-instruction full-scale IQ')
h.set_ylabels('Post-instruction characteristic path length')
#h.savefig(join(fig_dir, 'pre_prXpre_eff_inf-m.png'), dpi=300)

# FCI and PRIQ

In [ ]:
h = sns.lmplot('PRI2', 'post phys fci shen cpl', data=df_f, hue='Mod', 
           palette=f_2, legend_out=True, height=10.)

h.set_xlabels('Post-instruction perceptual reasoning')
h.set_ylabels('Post-instruction characteristic path length')
h.savefig(join(fig_dir, 'FCI_phys_post-cplXpost-priq_f.png'), dpi=300)

In [ ]:
h = sns.lmplot('PRI2', 'post phys fci shen cpl', data=df_m, hue='Mod', 
           palette=m_2, legend_out=True, height=10.)

h.set_xlabels('Post-instruction perceptual reasoning')
h.set_ylabels('Post-instruction characteristic path length')
h.savefig(join(fig_dir, 'FCI_phys_post-cplXpost-priq_m.png'), dpi=300)

# FCI and FSIQ

In [ ]:
h = sns.lmplot('FSIQ2', 'post phys fci shen cpl', data=df_m, hue='Mod', 
           palette=m_2, legend_out=True, height=10.)

h.set_xlabels('Post-instruction full-scale IQ')
h.set_ylabels('Post-instruction characteristic path length')
h.savefig(join(fig_dir, 'FCI_phys_post-cplXpost-fsiq_m.png'), dpi=300)

In [ ]:
h = sns.lmplot('FSIQ2', 'post phys fci shen cpl', data=df_f, hue='Mod', 
           palette=f_2, legend_out=True, height=10.)

h.set_xlabels('Post-instruction full-scale IQ')
h.set_ylabels('Post-instruction characteristic path length')
h.savefig(join(fig_dir, 'FCI_phys_post-cplXpost-fsiq_f.png'), dpi=300)

In [ ]:
h = sns.lmplot('deltaFSIQ', 'post phys fci shen eff', data=df_m, hue='Mod', 
           palette=m_2, legend_out=True, height=10.)

h.set_xlabels('Change in full-scale IQ')
h.set_ylabels('Post-instruction network efficiency')
h.savefig(join(fig_dir, 'FCI_phys_post-effXdelta-fsiq_m.png'), dpi=300)

In [ ]:
h = sns.lmplot('deltaPRI', 'post phys fci shen eff', data=df_m, hue='Mod', 
           palette=m_2, legend_out=True, height=10.)

h.set_xlabels('Change in perceptual reasoning')
h.set_ylabels('Post-instruction network efficiency')
h.savefig(join(fig_dir, 'FCI_phys_post-effXdelta-priq_m.png'), dpi=300)

In [ ]:
h = sns.lmplot('deltaFSIQ', 'post phys fci shen eff', data=df_f, hue='Mod', 
           palette=f_2, legend_out=True, height=10.)

h.set_xlabels('Change in full-scale IQ')
h.set_ylabels('Post-instruction network efficiency')
h.savefig(join(fig_dir, 'FCI_phys_post-effXdelta-fsiq_f.png'), dpi=300)

In [ ]:
h = sns.lmplot('deltaPRI', 'post phys fci shen eff', data=df_f, hue='Mod', 
           palette=f_2, legend_out=True, height=10.)

h.set_xlabels('Change in perceptual reasoning')
h.set_ylabels('Post-instruction network efficiency')
h.savefig(join(fig_dir, 'FCI_phys_post-effXdelta-priq_f.png'), dpi=300)

# Reasoning and WMIQ

In [ ]:
h = sns.lmplot('WMI1', 'delta infr reas shen eff', data=df_f, hue='Mod', 
           palette=f_2, legend_out=True, height=10.)

h.set_xlabels('Pre-instruction working memory')
h.set_ylabels('Change in network efficiency')
h.savefig(join(fig_dir, 'REAS_infr_delta-effXpre-wmiq_f.png'), dpi=300)

In [ ]:
h = sns.lmplot('WMI1', 'delta infr reas shen eff', data=df_m, hue='Mod', 
           palette=m_2, legend_out=True, height=10.)

h.set_xlabels('Pre-instruction working memory')
h.set_ylabels('Change in network efficiency')
h.savefig(join(fig_dir, 'REAS_infr_delta-effXpre-wmiq_m.png'), dpi=300)

In [ ]:
import seaborn as sns
sns.set()